In [8]:
import pandas as pd
import numpy as np

In [4]:
df=pd.read_csv('~/Desktop/ml-latest-small/ratings.csv',sep=',')
df_id=pd.read_csv('~/Desktop/ml-latest-small/links.csv',sep=',')

In [5]:
df = pd.merge(df, df_id, on = ['movieId'])

In [45]:
rating_matrix = np.zeros((df.userId.unique().shape[0],max(df.movieId)))

In [46]:
rating_matrix.shape

(610, 193609)

In [22]:
df

,userId,movieId,rating,timestamp,imdbId,tmdbId
0,1,1,4.0,964982703,114709,862.0
1,5,1,4.0,847434962,114709,862.0
2,7,1,4.5,1106635946,114709,862.0
3,15,1,2.5,1510577970,114709,862.0
4,17,1,4.5,1305696483,114709,862.0
5,18,1,3.5,1455209816,114709,862.0
6,19,1,4.0,965705637,114709,862.0
7,21,1,3.5,1407618878,114709,862.0
8,27,1,3.0,962685262,114709,862.0
9,31,1,5.0,850466616,114709,862.0


In [38]:
max(df.movieId)

193609

In [47]:
for row in df.itertuples():
    rating_matrix[row[1]-1, row[2]-1]=row[3]
rating_matrix=rating_matrix[:,:9000]

In [54]:
rating_matrix.shape

(610, 9000)

In [51]:
sparsity=float(len(rating_matrix.nonzero()[0]))
sparsity/=(rating_matrix.shape[0]*rating_matrix.shape[1])
sparsity*=100

In [52]:
sparsity

1.4194353369763206

In [55]:
train_matrix=rating_matrix.copy()
test_matrix=np.zeros(rating_matrix.shape)
for i in range(rating_matrix.shape[0]):
    rating_idx=np.random.choice(
    rating_matrix[i, :].nonzero()[0],
    size=10,
    replace=True)
train_matrix[i, rating_idx]=0.0
test_matrix[i, rating_idx]=rating_matrix[i, rating_idx]

In [56]:
similarity_user=train_matrix.dot(train_matrix.T)+1e-9
norms=np.array([np.sqrt(np.diagonal(similarity_user))])
similarity_user=(similarity_user/(norms*norms.T))
similarity_movie=train_matrix.T.dot(train_matrix)+1e-9
norms=np.array([np.sqrt(np.diagonal(similarity_movie))])
similarity_movie=(similarity_movie/(norms*norms.T))

In [62]:
from sklearn.metrics import mean_squared_error
prediction=similarity_user.dot(train_matrix)/np.array([np.abs(similarity_user).sum(axis=1)]).T
prediction=prediction[test_matrix.nonzero()].flatten()
test_vector=test_matrix[test_matrix.nonzero()].flatten()
mse=mean_squared_error(prediction, test_vector)
print('MSE ='+str(mse))

MSE =11.892092406
